In [ ]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import networkx as nx

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000022AABA45210>>
Traceback (most recent call last):
  File "E:\Felpipe\Proyectos propios\GNNs-Challenges\venv\lib\site-packages\ipykernel\ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
edges_df = pd.read_csv('musae_git_edges.csv')
#features_df = pd.read_json('musae_git_features.json')
with open('musae_git_features.json', 'r', encoding='utf-8') as data:
    features_raw = json.load(data)
target_df = pd.read_csv('musae_git_target.csv')

# EDA

## Descripción

In [ ]:
edges_df.head()

In [ ]:
features_raw['0']

In [ ]:
target_df.head()

In [ ]:
G = nx.Graph()
G.add_edges_from([(1, 2), (2, 3), (3, 1)])
G.add_edges_from([(2, 4), (4, 3)])
plt.axis('off')
nx.draw_networkx(G,node_size=600,font_color='white')
                 

In [ ]:
G = nx.from_pandas_edgelist(
    edges_df,
    source='id_1',
    target='id_2',
    create_using=nx.Graph()
)

In [ ]:
nx.set_node_attributes(G, features_raw)

In [ ]:
#Matriz de adyacencia
A_sparse = nx.adjacency_matrix(G)
#A = A_sparse.toarray() #Versión densa

In [ ]:
#Matriz de grado
from scipy.sparse import diags
nodelist = list(G.nodes())
deg = [G.degree(n) for n in nodelist]
D_sparse = diags(deg, offsets=0, format="csr")
# D = np.diag(deg) ##Versión densa

In [ ]:
#Matriz Laplaciana
L_sparse = nx.laplacian_matrix(G, nodelist=nodelist)

In [ ]:
#Resumen del grafo
N = G.number_of_nodes()
M = G.number_of_edges()
density = nx.density(G)
trans = nx.transitivity(G)
print('G = (V,E)')
print(f'Tamaño |V| = {N} nodos')
print(f'Tamaño |E| = {M} aristas')
print(f'Densidad: {density:.4f}')
print(f'Transitividad: {trans:.4f}')
degrees = np.array([d for _, d in G.degree()])
print(f'Mínimo grado: {degrees.min()}')
print(f'Máximo grado: {degrees.max()}')
print(f'Grado promedio: {degrees.mean():.2f}')
print(f'Número de componentes conexas: {nx.number_connected_components(G)}')

In [ ]:
largest_cc = max(nx.connected_components(G), key=len)
G_lcc = G.subgraph(largest_cc)
clustering_global = nx.transitivity(G) 

In [ ]:
print(f'Diámetro: {nx.diameter(G_lcc)}')
print(f'Radio: {nx.radius(G_lcc)}')

In [ ]:
print(f'Media del camino más corto: {nx.average_shortest_path_length(G_lcc)}')
print(f'Euleriano: {nx.is_eulerian(G)}')

In [ ]:
print(f'Grado centralidad: {nx.degree_centrality(G)}')
print(f'Cercanía centralidad: {nx.is_eulerian(G)}')
print(f'Entremedio del centro: {nx.betweenness_centrality(G)}')
print(f'Centralidad de los vectores propios: {nx.eigenvector_centrality(G)}')
print(f'Pagerank: {nx.pagerank(G)}')

In [ ]:
# 6. Densidad y transitividad
density     = nx.density(G)
transitivity = nx.transitivity(G)

# 7. Propiedades espectrales
A = nx.adjacency_matrix(G)
# – Radio espectral (mayor valor absoluto de A)
spectral_radius = eigsh(A, k=1, which='LM', return_eigenvectors=False)[0]

# – Algebraic connectivity (segundo menor autovalor de la Laplaciana)
L = nx.laplacian_matrix(G)
eigvals_L = eigsh(L, k=2, which='SM', return_eigenvectors=False)
fiedler_value = eigvals_L[1]

# – Multiplicidad del autovalor cero en L
zero_mult = sum(abs(eigvals_L) < 1e-6)

# – Bipartito (comprobación espectral más sencilla via función directa)
is_bipartite = nx.is_bipartite(G)

# – Número de caminatas cerradas de longitud 3 (triángulos × 6)
closed_walks_3 = (A ** 3).diagonal().sum().item()

# --- Mostrar resultados ---
print(f"Nodos: {num_nodes}, Aristas: {num_edges}")
print(f"Grado medio: {avg_degree:.2f}")
print(f"Componentes conexas: {num_cc}")
print(f"Diámetro (LCC): {diameter}, Radio (LCC): {radius}")
print(f"Clustering global: {clustering_global:.4f}")
print(f"Camino medio (LCC): {avg_shortest_path:.4f}")
print(f"Euleriano: {is_eulerian}")
print(f"Densidad: {density:.6f}, Transitividad: {transitivity:.4f}")
print(f"Radio espectral: {spectral_radius:.4f}")
print(f"Conectividad algebraica (Fiedler): {fiedler_value:.4f}")
print(f"Multiplicidad de 0 en L: {zero_mult}")
print(f"Bipartito: {is_bipartite}")
print(f"Caminatas cerradas de longitud 3: {closed_walks_3:.0f}")

In [ ]:
plt.axis('off')
nx.draw_networkx(G,node_size=600,font_color='white')

## Análisis de features

In [ ]:
label_counts = target_df['ml_target'].value_counts(normalize=True)
print(f'Balance de clases: {label_counts}')
plt.bar(['Web', 'ML'], [label_counts[0].item(), label_counts[1].item()])
plt.title('Label balances for class')
plt.xlabel('Labels')
plt.ylabel('Percentaje')
plt.show()

In [ ]:
from collections import Counter

lengths = []
counter = Counter()

for node, feat in features_raw.items():
    lengths.append(len(feat))
    counter.update(feat)

plt.hist(lengths, bins=30)
plt.title("Longitud del vector de features por nodo")
plt.xlabel("Número de tokens") 
plt.ylabel("Nodos")
plt.show()

print("Longitud media:", sum(lengths)/len(lengths))
print("Vocabulario total (tokens únicos):", len(counter))

## Features vs etiqueta objetivo

In [ ]:
df = pd.DataFrame({
    'node': list(map(int, features_raw.keys())),
    'feat_len': lengths
}).merge(target_df, how='left', left_on='node', right_on='id')

sns.boxplot(x='ml_target', y='feat_len', data=df)
plt.title("Longitud de features por clase")
plt.xlabel("Clase (0=Web,1=ML)"); plt.ylabel("Longitud vector")
plt.show()

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
#!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
!pip install -q torch-geometric

# Modelamiento

Este dataset no tiene mucha explicación, pero se nota algún tipo de bolsa de palabras en los números enteros (¿repositorios populares?, etc). Se pueden utilizar técnicas de NLP como word2vec para codificar esas palabras, o bien nn.Embeddings de pytorch

In [ ]:
!pip install scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy import sparse
from sklearn.preprocessing import MultiLabelBinarizer
from torch_geometric.data import Data

In [ ]:
edges = torch.tensor(edges_df[['id_1','id_2']].values.T, dtype=torch.long)
mlb = MultiLabelBinarizer(sparse_output=True)
feat_list = [features_raw[str(i)] for i in range(N)]
X_sparse = mlb.fit_transform(feat_list)
y = torch.tensor(target_df.sort_values('id')['ml_target'].values, dtype=torch.long)
data = Data(x=torch.tensor(X_sparse.toarray(), dtype=torch.float),
            edge_index=edges, y=y)

In [ ]:
# feat_list: lista de listas de enteros (tus tokens por nodo)
feat_list = [features_raw[str(i)] for i in range(N)]

# 1) aplanamos todas las listas en un solo vector
flat_tokens = [tok for sub in feat_list for tok in sub]

# 2) calculamos los offsets (punteros) donde empieza cada sublista
offsets = [0]
for sub in feat_list[:-1]:
    offsets.append(offsets[-1] + len(sub))

# 3) convertimos a tensores
data.token_ids  = torch.tensor(flat_tokens, dtype=torch.long)
data.token_ptrs = torch.tensor(offsets,    dtype=torch.long)

In [ ]:
def accuracy(y_pred, y_true):
    """Calculate accuracy."""
    return torch.sum(y_pred == y_true) / len(y_true)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, emb_dim, hidden_channels, out_channels,
                 vocab_size=4005):
        super().__init__()
        # 1) Capa de embedding “bag of tokens”
        self.embedding = torch.nn.EmbeddingBag(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            mode='mean',
            sparse=False
        )
        # 2) Convoluciones GCN
        self.conv1 = GCNConv(emb_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x = self.embedding(data.token_ids, data.token_ptrs)
        x = self.conv1(x, data.edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)

    def fit(self, data, epochs, epoch_review=20):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
                                     lr=0.01,
                                     weight_decay=5e-4)
        self.train()
        for epoch in range(1, epochs+1):
            optimizer.zero_grad()
            out = self(data)
            loss = criterion(out[data.train_mask],
                             data.y[data.train_mask])
            acc  = accuracy(out[data.train_mask].argmax(dim=1),
                            data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            if epoch % epoch_review == 0:
                self.eval()
                out = self(data)
                val_loss = criterion(out[data.val_mask],
                                     data.y[data.val_mask])
                val_acc  = accuracy(out[data.val_mask].argmax(dim=1),
                                    data.y[data.val_mask])
                print(f"Epoch {epoch:>3} | "
                      f"Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | "
                      f"Val Loss: {val_loss:.3f} | Val Acc: {val_acc*100:>5.2f}%")
                self.train()

    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data)
        acc = accuracy(out.argmax(dim=1)[data.test_mask],
                       data.y[data.test_mask])
        return acc


In [ ]:
gcn = GCN(N, 16, 2)
gcn.fit(data, epochs=1, epoch_review=1)
acc = gcn.test(data)
print(f'\nGCN test accuracy: {acc*100:.2f}%\n')

In [17]:
all_words = []
for key, value in datos.items():
    all_words += value

In [21]:
print(len(all_words))
print(np.unique(all_words))
print(len(np.unique(all_words)))

690374
[   0    1    2 ... 4002 4003 4004]
4005


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [24]:
len(target_df)

37700

In [44]:
M

289003